# Full MARS demo

This is the tutorial from the readme of MARS_Developer in Jupyter notebook form.

## 1. 📁 Create a new MARS Training Project

Your MARS project directory will contain all the files created during the process of training MARS detector and pose models on your data.

In [8]:
from create_new_project import *
# TODO: find less clunky way to call these scripts from Jupyter

location = 'K:/'
name = 'my_project'

create_new_project(location, name)

Project my_project created successfully.


**TODO** provide instructions on how to set up the fields of project_config.yaml.

## 2. ✍️ Collect a set of manually annotated animal poses.

#### 2.1 Extract video frames that you would like to annotate.
First, we need to collect a set of video frames to annotate. The script `extract_frames.py` will sample frames from all videos found in a directory, and save those frames as jpg files.

In [7]:
# python extract_raw_frames.py input_dir /path/to/videodir project /path/to/savedir/my_project n_frames 500
# TODO: wrap and test this; add Util to MARS_Developer

You should now have a folder `my_project/annotation_data` containing a directory `raw_images` of video frames.

#### 2.2 Run a labeling job.
Refer to [these instructions](https://github.com/neuroethology/MARS_Developer/blob/develop/pose_annotation_tools/docs/readme_groundTruthSetup.md) to run a labeling job on Amazon SageMaker.

This job will produce a file called `output.manifest` which you should add to `my_project/annotation_data`. If you change the name of this manifest file, be sure to update the `manifest_name` field of `my_project/project_config.yaml` appropriately.

#### 2.3 Post-process the manual pose annotations.
This will create "ground truth" keypoint locations by taking the median across AWS workers, and correct for left/right flips of body part labels. The processed annotation data will be added as a new file to `annotation_data`.

In [1]:
from pose_annotation_tools.json_util import *

project_path = 'K:\my_project'

manifest_to_dict(project_path)

Processing manifest file...
  frame 1000
  frame 2000
  frame 3000
  frame 4000
  frame 5000
Ground-truth keypoint locations extracted!


## 3. 🎯 Fine-tune the MARS detector to your data.